In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [13]:
#mol_str = """
#0 1
#O
#H 1 1.0
#H 1 1.0 2 104.0
#symmetry c1
#"""
mol_str = """
Li
H 1 1.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)

cavity_options = {
    'omega_value' : 0.36,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'nact_orbs' : 0,
    'nact_els' : 0
}


psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [14]:

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)



Start SCF iterations:

Canonical RHF One-electron energy = -13.3346745238277542
CQED-RHF One-electron energy      = -13.3241454366046739
Nuclear repulsion energy          = 1.5875316320100001
Dipole energy                     = 0.0100502927109982
SCF Iteration   1: Energy = -7.7639405850266137   dE = -7.76394E+00   dRMS = 4.20589E-04
SCF Iteration   2: Energy = -7.7639495413056760   dE = -8.95628E-06   dRMS = 9.34521E-05
SCF Iteration   3: Energy = -7.7639504939200652   dE = -9.52614E-07   dRMS = 4.41580E-05
SCF Iteration   4: Energy = -7.7639507470372031   dE = -2.53117E-07   dRMS = 2.31702E-05
SCF Iteration   5: Energy = -7.7639508179290075   dE = -7.08918E-08   dRMS = 1.22670E-05
SCF Iteration   6: Energy = -7.7639508378255808   dE = -1.98966E-08   dRMS = 6.49909E-06
SCF Iteration   7: Energy = -7.7639508434096260   dE = -5.58405E-09   dRMS = 3.44309E-06
SCF Iteration   8: Energy = -7.7639508449766854   dE = -1.56706E-09   dRMS = 1.82398E-06
SCF Iteration   9: Energy = -7.763950845

In [27]:
def compute_excitation_level(ket, ndocc):
    level = 0
    homo = ndocc - 1
    for i in range(0, len(ket)):
        if ket[i] > homo:
            level += 1
    return level


cis_tuple = H2_PF.generateCISTuple()
print(cis_tuple)

for alpha in cis_tuple:
    alpha_ex_level = compute_excitation_level(alpha, H2_PF.ndocc)

    for beta in cis_tuple:
        beta_ex_level = compute_excitation_level(beta, H2_PF.ndocc)
        if alpha_ex_level + beta_ex_level <= 1:
            e = Determinant(alphaObtList=alpha, betaObtList=beta)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5)]


In [48]:
#print(e)
alpha1 = (0,1,2,3)
beta1  = (0,1,2,3)

alpha2 = (0,1,2,4)
beta2 = (0,1,2,3)

det1 = Determinant(alphaObtList=alpha1, betaObtList=beta1)
det2 = Determinant(alphaObtList=alpha2, betaObtList=beta2)

print(det1)
print(det2)

numUniqueOrbitals = det1.numberOfTotalDiffOrbitals(det2)
print(numUniqueOrbitals)

|[0, 1, 2, 3][0, 1, 2, 3]>
|[0, 1, 2, 4][0, 1, 2, 3]>
1.0


In [49]:
((a1, b1), (a2, b2), sign, )= det1.getUniqueOrbitalsInListsPlusSign(det2)

print(a1, b1)
print(a2, b2)

mdet1 = Determinant.mixIndexList(a1, b1)
mdet2 = Determinant.mixIndexList(a2, b2)

print(mdet1)
print(mdet2)

[3] []
[4] []
[6]
[8]


In [6]:



#print(H2_PF.CISdets[0]) #.getUniqueOrbitalsInListsPlusSign(H2_PF.CISdets[0]))
#for i in range(20):
#    print(i,H2_PF.CISdets[i])
_bra = H2_PF.CISdets[11].copy()
_ket = H2_PF.CISdets[1].copy()

_bra.getUniqueOrbitalsInListsPlusSign
print(_bra)
print(_ket)
#_ket.removeBetaOrbital(1)
#_bra.removeAlphaOrbital(1)
#print(_ket)
#print(_bra)
#u1, u2, sign = _bra.getUniqueOrbitalsInListsPlusSign(_ket)
#print(sign)

AttributeError: 'PFHamiltonianGenerator' object has no attribute 'CISdets'

In [ ]:
def computeSplusSminusPQ(bra, ket, p, q):
    # act on ket with S-(q)
    # S-(q) kills q_alpha and creates q_beta
    a, b = ket.getOrbitalIndexLists()
    print(a, b)

    # check to see if q_alpha is unnoccupied
    if (q not in a):
        return 0
    else:
        ket.removeAlphaOrbital(q)
    if (q not in b):
        ket.addBetaOrbital(q)
    else:
        return 0
    
    # act on updated ket with S+(p)
    # S+(p) kills p_beta and creates p_alpha
    
    # get updated occupation list
    a, b = ket.getOrbitalIndexLists()
    print(a, b)
    if (p not in b):
        return 0
    else:
        ket.removeBetaOrbital(p)
    if (p not in a):
        ket.addAlphaOrbital(p)
    else:
        return 0
    
    numUniqueOrbitals = bra.numberOfTotalDiffOrbitals(ket)
    print("numUnique",numUniqueOrbitals)
    if numUniqueOrbitals==0:
        u1, u2, sign = bra.getUniqueOrbitalsInListsPlusSign(ket)
        print(u1, u2, sign)
        return sign
    else:
        return 0
    

    

In [ ]:
print(computeSplusSminusPQ(_bra, _ket, 5, 4))


In [ ]:
_H_spin = np.einsum("uj,vi,uv", H2_PF.Ca, H2_PF.Ca, H2_PF.H_1e_ao)
print(_H_spin)

In [ ]:
_H_spin = np.repeat(_H_spin, 2, axis=0)
print(_H_spin)

In [ ]:
_H_spin = np.repeat(_H_spin, 2, axis=1)
print(_H_spin)

In [ ]:
spin_ind = np.arange(_H_spin.shape[0], dtype=int) % 2
print(spin_ind)

In [ ]:
Hspin = _H_spin * (spin_ind.reshape(-1, 1) == spin_ind)
print(spin_ind.reshape(-1, 1) == spin_ind)
print(Hspin)

In [ ]:
        if self.ignore_coupling == False:
            self.H_1e_ao += self.q_PF_ao + self.d_PF_ao
        # build H_spin
        # spatial part of 1-e integrals
        _H_spin = np.einsum("uj,vi,uv", self.Ca, self.Ca, self.H_1e_ao)
        _H_spin = np.repeat(_H_spin, 2, axis=0)
        _H_spin = np.repeat(_H_spin, 2, axis=0)
        # spin part of 1-e integrals
        spin_ind = np.arange(_H_spin.shape[0], dtype=int) % 2
        # product of spatial and spin parts
        self.Hspin = _H_spin * (spin_ind.reshape(-1, 1) == spin_ind)

In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

# davidson eigenvalues and eigenvectors are stored in attributes
# H2_PF.cis_e and H2_PF.cis_c for now!


In [ ]:
print("checking to see if the Davidson Roots Agree!")
print(e_fci[:14])
print(H2_PF.cis_e[:14] )
print(np.allclose(e_fci[:14] , H2_PF.cis_e[:14]))


In [ ]:

H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




In [ ]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)